In [49]:
from hlsim import utils, hamiltonian_sim
import numpy as np
from pytket import Circuit
from pytket.passes import DecomposeBoxes
from pytket.circuit.display import render_circuit_jupyter as draw
from pytket.extensions.qiskit import tk_to_qiskit
from pytket.extensions.qiskit import AerStateBackend
import time

def build_xxz_trotter_circuit(
    n_qubits: int,
    total_time: float,
    Delta_ZZ: float,
    n_trotter_steps: int = 1,
    order: str = "first",
):
    """
    Constructs a Trotterized circuit for XXZ Hamiltonian time evolution.

    Returns:
    - circ_original : Circuit -> The Trotterized quantum circuit
    - circ_decomposed : Circuit -> Decomposed version showing full gate breakdown
    """
    # 1) Build the XXZ Hamiltonian
    H_xxz = hamiltonian_sim.get_xxz_chain_hamiltonian(n_qubits, Delta_ZZ)

    # 2) Choose first- or second-order Trotterization
    if order.lower() == "first":
        trotter_box = hamiltonian_sim.get_first_order_trotterization(
            hamiltonian=H_xxz,
            n_qubits=n_qubits,
            t_trotterization=total_time,
            n_trotter_steps=n_trotter_steps
        )
    elif order.lower() == "second":
        trotter_box = hamiltonian_sim.get_second_order_trotterization(
            hamiltonian=H_xxz,
            n_qubits=n_qubits,
            t_trotterization=total_time,
            n_trotter_steps=n_trotter_steps
        )
    else:
        raise ValueError("order must be 'first' or 'second'.")

    # 3) Embed the CircBox into a new Circuit
    circ_original = Circuit(n_qubits, name="Trotterized XXZ")
    circ_original.add_circbox(trotter_box, range(n_qubits))

    # 4) Decompose high-level circuit boxes for debugging
    circ_decomposed = circ_original.copy()
    DecomposeBoxes().apply(circ_decomposed)  # Unpacks CircBoxes

    return circ_original, circ_decomposed


##############################
# 2) EXAMPLE USAGE (MAIN)
##############################
if __name__ == "__main__":
    start_time = time.time()
    # Define Parameters
    N = 4          # Number of qubits (spin chain size)
    t = 0.01          # Total evolution time
    Delta = 1.0      # Anisotropy parameter
    steps = 2      # Number of Trotter steps
    trotter_order = "first"  # Use "second" for second-order Trotter

    # Generate Circuit
    circ_original, circ_decomposed = build_xxz_trotter_circuit(
        n_qubits=N,
        total_time=t,
        Delta_ZZ=Delta,
        n_trotter_steps=steps,
        order=trotter_order
    )


    sv_backend = AerStateBackend()
    compiled_circuit = sv_backend.get_compiled_circuit(circ_original)


    result = sv_backend.run_circuit(compiled_circuit)
    statevector = result.get_state()


    print("Statevector dimensions: ", statevector.shape)
    print("GHZ Statevector: ", statevector)

    probabilities = np.abs(statevector) ** 2
    print("Probabilities:", probabilities)
    print("Sum of probabilities:", np.sum(probabilities))  # Should be ~1.0
    end_time = time.time()

    print(f"RUNTIME: {end_time-start_time}")


    # print("=== Original Trotter Circuit ===")
    # draw(circ_decomposed)  # This should now be a proper pytket Circuit

    # # print("\n=== Decomposed Circuit ===")
    # # print(circ_decomposed.get_ascii())  # ASCII-based text visualization

    # # If using Jupyter, visualize with draw()
    # draw(circ_decomposed)

    # # Convert to Qiskit for a detailed visualization
    # qiskit_circ = tk_to_qiskit(circ_decomposed)
    # print("\n=== Qiskit Text Drawer ===")
    # print(qiskit_circ.draw("text"))  # ASCII Qiskit circuit


Statevector dimensions:  (16,)
GHZ Statevector:  [ 9.99550034e-01-2.99955002e-02j -1.63064007e-16+1.47911420e-31j
 -1.14491749e-16+1.66533454e-16j  5.78704492e-17+5.55000494e-17j
 -5.64251977e-17-3.92523115e-17j  6.00907472e-17-5.55000494e-17j
 -1.17756934e-16-2.69859641e-17j  1.54074396e-32+0.00000000e+00j
  3.81639165e-17+7.28583860e-17j  1.73472348e-17+1.73472348e-17j
  3.91179502e-17-3.64195945e-17j -7.95780251e-17-2.53941961e-17j
 -1.36262465e-16+5.72362762e-17j  3.10057678e-17-2.36137954e-18j
 -3.81639165e-17+4.51028104e-17j  5.89805982e-17+2.42861287e-17j]
Probabilities: [1.00000000e+00 2.65898703e-32 4.08417519e-32 6.42924437e-33
 4.72454689e-33 6.69115338e-33 1.45949379e-32 2.37389194e-64
 6.76482893e-33 6.01853108e-34 2.85660089e-33 6.97752727e-33
 2.18434506e-32 9.66933748e-34 3.49074802e-33 4.06852701e-33]
Sum of probabilities: 0.9999999999999987
RUNTIME: 0.40396618843078613


In [ ]:
from pytket.extensions.qiskit import AerStateBacken

sv_backend = AerStateBackend()

ghz_state = prepare_ghz(n_qubits=4)

result = sv_backend.run_circuit(ghz_state)
statevector = result.get_state()


print("Statevector dimensions: ", statevector.shape)
print("GHZ Statevector: ", statevector)
